# 1. Setup

In [1]:
import os 
os.chdir('..')
os.getcwd()

'e:\\Data Driven mkt\\Midterm 2'

In [2]:
import numpy as np
import pandas as pd
from typing import List, Union, Iterable
import re, string, math

In [3]:
files = [f for f in os.listdir('Data/raw') if f.startswith('reviews_')]
dfs = []
for f in files:
    df = pd.read_csv(f'Data/raw/{f}', low_memory=False)
    dfs.append(df)
df = pd.concat(dfs, ignore_index=True)

# 2. Cleaning

In [4]:
df.rename(columns={'Unnamed: 0': 'review_id'}, inplace=True)
df['review_id'] = range(len(df))

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1094411 entries, 0 to 1094410
Data columns (total 19 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   review_id                 1094411 non-null  int64  
 1   author_id                 1094411 non-null  object 
 2   rating                    1094411 non-null  int64  
 3   is_recommended            926423 non-null   float64
 4   helpfulness               532819 non-null   float64
 5   total_feedback_count      1094411 non-null  int64  
 6   total_neg_feedback_count  1094411 non-null  int64  
 7   total_pos_feedback_count  1094411 non-null  int64  
 8   submission_time           1094411 non-null  object 
 9   review_text               1092967 non-null  object 
 10  review_title              783757 non-null   object 
 11  skin_tone                 923872 non-null   object 
 12  eye_color                 884783 non-null   object 
 13  skin_type                 9

## Numerical columns

In [6]:
df.describe()

,review_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,price_usd
count,1.094411e+06,1.094411e+06,926423.000000,532819.000000,1.094411e+06,1.094411e+06,1.094411e+06,1.094411e+06
mean,5.472050e+05,4.299158e+00,0.839962,0.767782,4.177126e+00,8.948695e-01,3.282257e+00,4.900838e+01
std,3.159294e+05,1.149444e+00,0.366642,0.317164,2.271524e+01,5.288943e+00,1.967482e+01,4.004338e+01
min,0.000000e+00,1.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00
25%,2.736025e+05,4.000000e+00,1.000000,0.652174,0.000000e+00,0.000000e+00,0.000000e+00,2.500000e+01
50%,5.472050e+05,5.000000e+00,1.000000,0.928571,0.000000e+00,0.000000e+00,0.000000e+00,3.900000e+01
75%,8.208075e+05,5.000000e+00,1.000000,1.000000,3.000000e+00,1.000000e+00,3.000000e+00,6.200000e+01
max,1.094410e+06,5.000000e+00,1.000000,1.000000,5.464000e+03,1.159000e+03,5.050000e+03,1.900000e+03


In [7]:
df.select_dtypes(include = 'number').isnull().sum()

review_id                        0
rating                           0
is_recommended              167988
helpfulness                 561592
total_feedback_count             0
total_neg_feedback_count         0
total_pos_feedback_count         0
price_usd                        0
dtype: int64

In [8]:
numeric_columns = df.select_dtypes(include=['number']).columns
numeric_columns = [col for col in numeric_columns if col != 'review_id']
df[numeric_columns].corr(numeric_only=True).T.style.background_gradient(cmap='RdGy', axis=1)

,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,price_usd
rating,1.000000,0.845274,0.183906,-0.088634,-0.176603,-0.054857,0.007105
is_recommended,0.845274,1.000000,0.138562,-0.088437,-0.160550,-0.058105,0.019731
helpfulness,0.183906,0.138562,1.000000,0.015220,-0.204209,0.072386,-0.052726
total_feedback_count,-0.088634,-0.088437,0.015220,1.000000,0.652809,0.979047,0.021480
total_neg_feedback_count,-0.176603,-0.160550,-0.204209,0.652809,1.000000,0.484872,0.021083
total_pos_feedback_count,-0.054857,-0.058105,0.072386,0.979047,0.484872,1.000000,0.019132
price_usd,0.007105,0.019731,-0.052726,0.021480,0.021083,0.019132,1.000000


In [9]:
df['is_recommended'].groupby(df['rating']).agg(['mean', 'std', 'count']).style.background_gradient(cmap='RdGy', axis=1)

,mean,std,count
rating,,,
1,0.007734,0.087605,52493
2,0.036675,0.187966,45262
3,0.356594,0.478997,71395
4,0.965680,0.182051,170192
5,0.998642,0.036820,587081


In [10]:
def create_sentiment_label(rating):
    if rating <= 2:
        return 0
    elif rating == 3:
        return 1
    else:
        return 2
    
df['sentiment']= df['rating'].apply(create_sentiment_label)

In [11]:
df.loc[df['sentiment'] == 2, 'is_recommended'] = df.loc[df['sentiment'] == 2, 'is_recommended'].fillna(1)
df.loc[df['sentiment'] == 0, 'is_recommended'] = df.loc[df['sentiment'] == 0, 'is_recommended'].fillna(0)
# Fill NaN values in 'is_recommended' for sentiment == 1 with the calculated mean
mean_sentiment_1 = df.loc[df['sentiment'] == 1, 'is_recommended'].mean()
df.loc[df['sentiment'] == 1, 'is_recommended'] = df.loc[df['sentiment'] == 1, 'is_recommended'].fillna(mean_sentiment_1)
df.loc[df['total_pos_feedback_count'] == 0, 'helpfulness'] = df.loc[df['total_pos_feedback_count'] == 0, 'is_recommended'].fillna(-1)

## Categorical columns

In [12]:
df.select_dtypes(include = 'object').describe()

,author_id,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name
count,1094411,1094411,1092967,783757,923872,884783,982854,867643,1094411,1094411,1094411
unique,503216,5317,969419,364105,14,6,4,7,2351,2334,142
top,1696370280,2020-06-11,I received this in a sample. I have alot of ac...,Love it!,light,brown,combination,brown,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,CLINIQUE
freq,292,2028,59,6125,266418,471181,544513,403099,16138,16138,49029


In [13]:
df[['review_text','review_title','author_id','product_id']]= df[['review_text','review_title','author_id','product_id']].drop_duplicates()

In [14]:
df.select_dtypes(include = 'object').isnull().sum()


author_id             489
submission_time         0
review_text          1932
review_title       311057
skin_tone          170539
eye_color          209628
skin_type          111557
hair_color         226768
product_id            489
product_name            0
brand_name              0
dtype: int64

In [15]:
# Create a mapping of (brand_name, product_name) to product_id for non-null product_id rows
product_id_mapping = (
    df.loc[df['product_id'].notna()]
    .groupby(['brand_name', 'product_name'])['product_id']
    .first()
    .to_dict()
)

# Fill missing product_id values by mapping based on brand_name and product_name
df.loc[df['product_id'].isna(), 'product_id'] = df.loc[df['product_id'].isna()].apply(
    lambda row: product_id_mapping.get((row['brand_name'], row['product_name'])),
    axis=1
)

In [16]:
def clean_data_v2(raw: Union[pd.Series, Iterable[str]]) -> List[str]:
    if isinstance(raw, pd.Series):
        raw_list = raw.tolist()
    else:
        raw_list = list(raw)

    # Compile regex patterns
    emoji_pattern = re.compile(
        "["               
        "\U0001F600-\U0001F64F"
        "\U0001F300-\U0001F5FF"
        "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF"
        "\U00002700-\U000027BF"
        "\U0001F900-\U0001F9FF"
        "\U00002600-\U000026FF"
        "\U000025A0-\U000025FF"
        "]+",
        flags=re.UNICODE,
    )
    url_pattern = re.compile(r'https?://\S+')
    punc_table = str.maketrans('', '', string.punctuation)

    cleaned: List[str] = []
    for text in raw_list:
        # None or NaN -> empty string
        if text is None or (isinstance(text, float) and math.isnan(text)):
            cleaned.append('')
            continue

        # Lowercase & cast to str
        s = str(text).lower()

        # Remove emojis, URLs
        s = emoji_pattern.sub('', s)
        s = url_pattern.sub('', s)

        # Collapse whitespace
        s = s.replace('\n', ' ').replace('\r', ' ')
        s = re.sub(r'\s+', ' ', s)

        # Remove punctuation & trim
        s = s.translate(punc_table).strip()

        cleaned.append(s)

    return cleaned


In [17]:
df["review_text"] = clean_data_v2(df["review_text"])

In [ ]:
# chunk_size = 250000  # Số dòng mỗi chunk
# for i in range(0, len(df), chunk_size):
#     mode = 'w' if i == 0 else 'a'  # Ghi đè (write) hoặc thêm (append)
#     header = i == 0  # Chỉ ghi header ở chunk đầu tiên
#     df.iloc[i:i+chunk_size].to_csv('Data/processsed/reviews_cleaned.csv', mode=mode, header=header, index=False)

## Train val test split

In [19]:
# Đảm bảo cột 'submission_time' là kiểu datetime
df['submission_time'] = pd.to_datetime(df['submission_time'], errors='coerce')
# Nhóm dữ liệu theo ngày (ngày-tháng-năm)
df['date'] = df['submission_time'].dt.date  # Tạo cột chỉ chứa ngày
grouped = df.groupby('date')
# Lấy mẫu ngẫu nhiên từ mỗi nhóm (10% hoặc ít nhất 1 mẫu)
sampled_data = grouped.apply(lambda x: x.sample(frac=0.05, random_state=42) if len(x) > 1 else x)
sampled_data.reset_index(drop=True, inplace=True)

print(f"Số lượng mẫu sau khi lấy: {len(sampled_data)}")
display(sampled_data)

C:\Users\nn200\AppData\Local\Temp\ipykernel_26808\1645073873.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_data = grouped.apply(lambda x: x.sample(frac=0.05, random_state=42) if len(x) > 1 else x)


Số lượng mẫu sau khi lấy: 54734


,review_id,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,...,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd,sentiment,date
0,891454,1117870798,5,1.0,0.285714,21,15,6,2008-08-29,it is cruel for nature to combine aging and ac...,...,light,NaN,combination,NaN,P7365,Pure Skin Clarifying Dietary Supplement,Murad,50.0,2,2008-08-29
1,24870,1221497549,4,1.0,1.000000,0,0,0,2008-09-03,love it i had to get used to it not foaming it...,...,light,NaN,dry,NaN,P7880,Soy Hydrating Gentle Face Cleanser,fresh,39.0,2,2008-09-03
2,780337,1229624109,5,1.0,1.000000,4,0,4,2008-09-04,this product is great i only have to use it oc...,...,mediumTan,NaN,combination,NaN,P122661,7 Day Face Scrub Cream Rinse-Off Formula,CLINIQUE,26.0,2,2008-09-04
3,622360,1526018747,5,1.0,0.111111,9,8,1,2008-09-06,this stuff works so well,...,NaN,NaN,NaN,NaN,P2046,Brumisateur Natural Mineral Water Facial Spray...,Evian,23.5,2,2008-09-06
4,478352,54172647,5,1.0,1.000000,0,0,0,2008-09-11,love it it’s moisturizing and helps keep conce...,...,lightMedium,NaN,combination,NaN,P174502,All About Eyes Rich Eye Cream,CLINIQUE,37.0,2,2008-09-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54729,597480,20086078659,5,1.0,1.000000,0,0,0,2023-03-21,this is the best vegan lip balm i have ever us...,...,NaN,blue,dry,black,P455936,Lip Butter Balm,Summer Fridays,24.0,2,2023-03-21
54730,869055,23046286666,5,1.0,1.000000,0,0,0,2023-03-21,i’m always looking for spf that don’t make me ...,...,tan,brown,combination,black,P504986,Urban Environment Vita-Clear Sunscreen SPF 42,Shiseido,38.0,2,2023-03-21
54731,978438,10829478865,5,1.0,1.000000,0,0,0,2023-03-21,i am honestly really impressed with this brand...,...,NaN,NaN,NaN,NaN,P475951,Green Tea Hyaluronic Acid Hydrating Serum,innisfree,30.0,2,2023-03-21
54732,1079588,8118994357,5,1.0,1.000000,0,0,0,2023-03-21,this is like a glass of water for your skin my...,...,light,brown,combination,blonde,P482695,Water Bank Blue Hyaluronic Gel Moisturizer,LANEIGE,40.0,2,2023-03-21


In [21]:
# Xác định tỷ lệ chia dữ liệu
train_ratio = 0.8
val_ratio = 0.1 
test_ratio = 0.1

# Tính số lượng mẫu cho mỗi tập
n = len(sampled_data)
train_end = int(n * train_ratio)
val_end = train_end + int(n * val_ratio)

# Chia dữ liệu
train_data = sampled_data.iloc[:train_end]
val_data = sampled_data.iloc[train_end:val_end]
test_data = sampled_data.iloc[val_end:]

# Kiểm tra kích thước các tập dữ liệu
print(f"Số lượng mẫu trong tập train: {len(train_data)}")
print(f"Số lượng mẫu trong tập validation: {len(val_data)}")
print(f"Số lượng mẫu trong tập test: {len(test_data)}")

# Lưu các tập dữ liệu nếu cần
train_data.to_csv('data/processsed/train_data.csv', index=False)
val_data.to_csv('data/processsed/val_data.csv', index=False)
test_data.to_csv('data/processsed/test_data.csv', index=False)

Số lượng mẫu trong tập train: 43787
Số lượng mẫu trong tập validation: 5473
Số lượng mẫu trong tập test: 5474
